In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# preprocessing
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# training
from tensorflow import keras
from keras import layers
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import cross_val_score

train_df = pd.read_csv('./training.csv')
test_df = pd.read_csv('./test.csv')

In [ ]:
image_list = []
for i in range(train_df['Image'].count()):
    image = train_df['Image'][i].split(' ')
    image_list.append(image)

print(image_list)

input_shape = len(image_list[0])

def create_model():
    model = keras.Sequential([
        layers.Dense(units=150, activation='relu', input_shape=[input_shape]),
        layers.Dense(units=50, activation='sigmoid'),
        layers.Dense(units=1)
    ])

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

wrapped_model = KerasRegressor(build_fn=create_model(), verbose=0)

process_pipeline = Pipeline(steps=[
        ('proprocessing', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
        ('model', wrapped_model)
    ])


In [ ]:
y_train = train_df[['Image']]
y_train = SimpleImputer(missing_values=np.nan, strategy='most_frequent').fit_transform(y_train)

image_list_array = np.array(image_list, dtype="float")
x_train = image_list_array.reshape(-1, 96, 96, 1)
# plt.imshow(train_x[5].reshape(96,96),cmap='gray')
# plt.show()

In [ ]:
num_samples = x_train.shape[0]
height = x_train.shape[1]
width = x_train.shape[2]
channels = x_train.shape[3]

x_train_flattened = x_train.reshape(num_samples, height * width * channels)

process_pipeline.fit(x_train_flattened, y_train)
scores = -1 * cross_val_score(x_train_flattened, y_train, cv=5, scoring='neg_mean_squared_error')
print(scores)